# Pre-process EBSD patterns from an Al-steel joint

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

See the relevant package documentation for more details on the packages used here:
* diffsims: https://diffsims.readthedocs.io/en/latest/
* kikuchipy: https://kikuchipy.org/en/stable/
* hyperspy: hyperspy.org/hyperspy-doc/current/
* pyebsdindex: https://pyebsdindex.readthedocs.io/en/latest/
* orix: https://orix.readthedocs.io/en/stable/

Import required packages and print their versions

In [23]:
# Replace "inline" with "qt5" from the pyqt package for interactive plotting
%matplotlib qt5

from datetime import date
import importlib_metadata
import os

import dask
from dask.diagnostics import ProgressBar
from diffpy.structure import Lattice, Structure
from diffsims.crystallography import ReciprocalLatticePoint
import hyperspy.api as hs
import kikuchipy as kp
import matplotlib.pyplot as plt
import numpy as np
from pyebsdindex import ebsd_index, pcopt
from orix import io, plot, sampling
from orix.crystal_map import CrystalMap, PhaseList
from orix.quaternion import Rotation


# Directories
dset = "a"
dir_mp = "/home/hakon/kode/emsoft/emdata/crystal_data"
dir_data = os.path.join("/home/hakon/phd/data/tina", dset)
dir_nordif = os.path.join(dir_data, "nordif")
dir_kp = os.path.join(dir_data, "kp")

# Data set specific parameters
# Dataset naming (a-c) = (I-III)
cal_pats_al = {"a": [0, 1, 2, 3], "b": [0, 1, 2, 3], "c": [0, 1, 2]}
pc0 = {"a": (0.42, 0.80, 0.51), "b": (0.42, 0.80, 0.51), "c": (0.41, 0.81, 0.52)}

# Matplotlib
plt.rcParams.update({"figure.figsize": (5, 5), "font.size": 12})
savefig_kw = dict(bbox_inches="tight", pad_inches=0, dpi=150)

print("Run date: ", date.today())
print("\nSoftware versions\n------------------")
for pkg in [
    "dask",
    "diffpy.structure",
    "diffsims",
    "hyperspy",
    "kikuchipy",
    "matplotlib",
    "numpy",
    "pyebsdindex",
    "orix",
]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

Run date:  2022-05-27

Software versions
------------------
dask : 2022.5.0
diffpy.structure : 3.0.1
diffsims : 0.4.2
hyperspy : 1.7.0
kikuchipy : 0.6.dev0
matplotlib : 3.5.2
numpy : 1.21.6
pyebsdindex : 0.1.dev1
orix : 0.9.0


# Pre-correction maps

Load data into memory

In [2]:
s = kp.load(os.path.join(dir_nordif, "Pattern.dat"), lazy=False)
sig_shape = s.axes_manager.signal_shape[::-1]
s

<EBSD, title: Pattern, dimensions: (148, 50|240, 240)>

Obtain a mean intensity map

In [3]:
s_mean = s.mean(axis=s.axes_manager.signal_axes)

Save unprocessed map and contrast stretched map

In [4]:
# Unprocessed map
map_mean1 = s_mean.data
plt.imsave(os.path.join(dir_kp, "maps_mean.png"), map_mean1, cmap="gray")

# Contrast stretched map
percentiles = np.percentile(map_mean1, q=(1, 99))
map_mean2 = kp.pattern.rescale_intensity(map_mean1, in_range=percentiles)
plt.imsave(os.path.join(dir_kp, "maps_mean_q1_q99.png"), map_mean2, cmap="gray")

Background correction:
1. Remove static background
2. Remove dynamic background
3. Average patterns with their eight nearest neighbour using a Gaussian kernel with $\sigma$ = 1

In [8]:
s.remove_static_background()

Removing the static background:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.6s


In [9]:
s.remove_dynamic_background()

Removing the dynamic background:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  6.6s


In [10]:
w = kp.filters.Window(window="gaussian", shape=(3, 3), std=1)
w.plot()

In [11]:
s.average_neighbour_patterns(window=w)

Averaging with the neighbour patterns:
[########################################] | 100% Completed |  7.4s


Save processed patterns to file

In [12]:
s.save(os.path.join(dir_kp, "pattern_sda.h5"))

0.005268482367197672 min


## Pre-indexing maps

Generate image quality $\mathbf{Q}$ and average dot product maps and save to
file

In [13]:
iq = s.get_image_quality()
plt.imsave(os.path.join(dir_kp, "maps_iq.png"), arr=iq, cmap="gray")

Calculating the image quality:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  4.7s


In [14]:
adp = s.get_average_neighbour_dot_product_map()
plt.imsave(os.path.join(dir_kp, "maps_adp.png"), arr=adp, cmap="gray")

Calculating average neighbour dot product map:
[########################################] | 100% Completed | 12.7s


## Projection center from PyEBSDIndex

Load calibration patterns

In [15]:
s_cal0 = kp.load(os.path.join(dir_nordif, "Setting.txt"))
sig_shape_cal = s_cal0.axes_manager.signal_shape[::-1]
#s_cal0.static_background = s.static_background.copy()
s_cal0

<EBSD, title: Calibration patterns, dimensions: (2|240, 240)>

Remove background

In [16]:
s_cal0.remove_static_background()
s_cal0.remove_dynamic_background()

Removing the static background:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
Removing the dynamic background:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


Identify Al and $\alpha$-Fe patterns, as `PyEBSDIndex` can only (Hough) index
cubic phases

In [17]:
s_cal0.plot(navigator=None)

Extract Al or $\alpha$-Fe patterns

In [18]:
s_cal = kp.signals.EBSD(s_cal0.data[cal_pats_al[dset]])
s_cal.axes_manager[0].name = "x"
nav_size = s_cal.axes_manager.navigation_size

Extract relevant metadata

In [19]:
md_sem = s_cal0.metadata.Acquisition_instrument.SEM
md_ebsd = md_sem.Detector.EBSD
sample_tilt = md_ebsd.sample_tilt  # Degrees
camera_tilt = md_ebsd.azimuth_angle  # Degrees
energy = md_sem.beam_energy  # kV

Generate an indexer instance with `PyEBSDIndex` for easy storage of relevant
parameters used in projection center (PC) optimization

In [20]:
indexer = ebsd_index.EBSDIndexer(
    phaselist=["BCC", "FCC"],
    vendor="EDAX",
    PC=None,
    sampleTilt=sample_tilt,
    camElev=camera_tilt,
    patDim=sig_shape_cal
)

Determine PC from single pattern using an initial guess, as a test

In [22]:
pc0_i = pc0[dset_no]
print(pc0_i)

pc = pcopt.optimize(s_cal.inav[0].data, indexer, PC0=pc0_i)
print(pc)

(0.42, 0.81, 0.51)
[0.42025143 0.80980777 0.51327385]


Determine PCs from all patterns

In [24]:
pcs = np.zeros((nav_size, 3))
for i in range(nav_size):
    pcs[i] = pcopt.optimize(s_cal.inav[i].data, indexer, PC0=pc0_i)
print(pcs)

pc = pcs.mean(axis=0)
print(pc)

[[0.42025143 0.80980777 0.51327385]
 [0.42365032 0.81203585 0.51243699]]
[0.42195087 0.81092181 0.51285542]


Index calibration patterns to check PCs and extract rotations

In [25]:
data = indexer.index_pats(
    patsin=s_cal.data,
    patstart=0,
    npats=-1,
    clparams=None,
    PC=pc,
)
rot = Rotation(data[0]["quat"][2])

Are the patterns Fe or Al?

In [26]:
phase_id = data[0]["phase"][2]
print(phase_id)

[0 0]


Separate rotations into those of Fe and those of Al

In [ ]:
rot_fe = rot[phase_id == 0]
#rot_al = rot[phase_id == 1]

Separate calibration patterns into those of Fe and those of Al

In [30]:
s_cal_fe = kp.signals.EBSD(s_cal.data[phase_id == 0]).squeeze()
#s_cal_al = kp.signals.EBSD(s_cal.data[phase_id == 1]).squeeze()

Describe the detector-sample geometry

In [28]:
detector = kp.detectors.EBSDDetector(
    shape=sig_shape_cal,
    pc=pc,  # Use average PC to reduce potential for single error upon refinement
    sample_tilt=sample_tilt,
    tilt=camera_tilt,
    convention="edax",
)
detector

EBSDDetector (240, 240), px_size 1 um, binning 1, tilt 0.0, azimuthal 0, pc (0.422, 0.189, 0.513)

### Inspect Fe PCs

Load Fe master pattern to use in PC refinement and to extract the Fe crystal structure

In [34]:
mp_fe = kp.load(
    os.path.join(dir_mp, "ferrite", "ferrite_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy,
    hemisphere="north",
)
mp_fe.phase.name = "ferrite"

Inspect geometrical simulations

In [35]:
rlp_fe = ReciprocalLatticePoint(phase=mp_fe.phase, hkl=((1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)))
rpl_fe = rlp_fe[rlp_fe.allowed]
rlp2_fe = rlp_fe.symmetrise()
simgen_fe = kp.generators.EBSDSimulationGenerator(detector=detector, phase=rlp_fe.phase, rotations=rot_fe)
geosim_fe = simgen_fe.geometrical_simulation(reciprocal_lattice_point=rlp2_fe)
markers_fe = geosim_fe.as_markers(pc=False, bands=True, zone_axes_labels=False, zone_axes=False)

#del s_cal_fe.metadata.Markers
s_cal_fe.add_marker(marker=markers_fe, plot_marker=False, permanent=True)
s_cal_fe.plot(navigator=None)

Refine results from `PyEBSDIndex`

In [36]:
s_cal_fe.axes_manager[0].name = "x"  # Will be unnecessary in kikuchipy v0.6

In [37]:
xmap_fe = CrystalMap(rotations=rot_fe, phase_list=PhaseList(mp_fe.phase))

# First refine orientations, then projection centers
ref_kwargs = dict(detector=detector, master_pattern=mp_fe, energy=energy)
xmap_fe_refined = s_cal_fe.refine_orientation(xmap=xmap_fe, **ref_kwargs)
_, detector_fe_ref = s_cal_fe.refine_projection_center(xmap=xmap_fe_refined, **ref_kwargs)

Refinement information:
	Local optimization method: Nelder-Mead (minimize)
	Keyword arguments passed to method: {'method': 'Nelder-Mead'}
Refining 2 orientation(s):
[########################################] | 100% Completed |  2.2s
Refinement speed: 0 patterns/s
Refinement information:
	Local optimization method: Nelder-Mead (minimize)
	Keyword arguments passed to method: {'method': 'Nelder-Mead'}
Refining 2 projection center(s):
[########################################] | 100% Completed |  0.8s
Refinement speed: 2 patterns/s


Check geometrical simulations of refined orientations and PCs

In [38]:
simgen_fe = kp.generators.EBSDSimulationGenerator(
    detector=detector_fe_ref, phase=mp_fe.phase, rotations=xmap_fe_refined.rotations,
)
geosim_fe = simgen_fe.geometrical_simulation(reciprocal_lattice_point=rlp2_fe)
markers_fe = geosim_fe.as_markers(pc=False, bands=True, zone_axes_labels=False, zone_axes=False)

del s_cal_fe.metadata.Markers
s_cal_fe.add_marker(marker=markers_fe, plot_marker=False, permanent=True)
s_cal_fe.plot(navigator=None)

### Inspect Al PCs

Load Al master pattern to use in PC refinement and to extract the Al crystal structure

In [ ]:
mp_al = kp.load(
    os.path.join(dir_mp, "al", "al_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy,
    hemisphere="north",
)

Inspect geometrical simulations

In [ ]:
rlp_al = ReciprocalLatticePoint(phase=mp_al.phase, hkl=((1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)))
rlp2_al = rlp_al.symmetrise()
simgen_al = kp.generators.EBSDSimulationGenerator(detector=detector, phase=rlp_al.phase, rotations=rot_al)
geosim_al = simgen_al.geometrical_simulation(reciprocal_lattice_point=rlp2_al)
markers_al = geosim_al.as_markers(pc=False, bands=True, zone_axes_labels=False, zone_axes=False)

#del s_cal_al.metadata.Markers
s_cal_al.add_marker(marker=markers_al, plot_marker=False, permanent=True)
s_cal_al.plot(navigator=None)

Refine results from `PyEBSDIndex`

In [ ]:
s_cal_al.axes_manager[0].name = "x"  # Will be unnecessary in kikuchipy v0.6

In [ ]:
xmap_al = CrystalMap(rotations=rot_al, phase_list=PhaseList(mp_al.phase))

# First refine orientations, then projection centers
ref_kwargs = dict(detector=detector, master_pattern=mp_al, energy=energy)
xmap_al_refined = s_cal_al.refine_orientation(xmap=xmap_al, **ref_kwargs)
_, detector_al_ref = s_cal_al.refine_projection_center(xmap=xmap_al_refined, **ref_kwargs)

Check geometrical simulations of refined orientations and PCs

In [ ]:
simgen_al = kp.generators.EBSDSimulationGenerator(
    detector=detector_al_ref, phase=mp_al.phase, rotations=xmap_al_refined.rotations,
)
geosim_al = simgen_al.geometrical_simulation(reciprocal_lattice_point=rlp2_al)
markers_al = geosim_al.as_markers(pc=False, bands=True, zone_axes_labels=False, zone_axes=False)

del s_cal_al.metadata.Markers
s_cal_al.add_marker(marker=markers_al, plot_marker=False, permanent=True)
s_cal_al.plot(navigator=None)

## Save PCs to file

In [39]:
np.savetxt(
    os.path.join(dir_kp, "cal_pcs.txt"),
    np.column_stack((
        cal_pats_al[dset],
        np.row_stack((
#            detector_al_ref.pc,
            detector_fe_ref.pc
        ))
    )),
    fmt="%i %.12f %.12f %.12f",
    header="Cal. pattern, PC (x, y, z) in Bruker's convention"
)